<a href="https://colab.research.google.com/github/IlyaZutler/Project_2-Trucks/blob/main/DM%20_%20Project_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic mitochondria Project - Heavy Machinery Auction Price Estimator

> https://www.kaggle.com/t/9baafb8850d74e4499c7b1ba97d6f115

### Timeline
- **Start Date:** [Start Date]
- **End Date:** 14/07/2024 (11 days to go)

### 2. Exploratory Data Analysis (EDA)


### 3. Data Preprocessing

### 6. Model Improvement

- Handle missing values and categorical variables more effectively.
- Use feature importances to identify key features.
- Perform feature engineering to create new informative features.
- Tune hyperparameters using grid search or other techniques.
- Monitor for overfitting by comparing training and testing performance.

## Practical Data Science Guidelines

- **Efficient Workflows:** Use a random subset of 20,000 rows for initial experiments. Use the full dataset for the final submission.
- **Iterative Approach:** Start with a basic model and iteratively improve it by trying small ideas.
- **Feature Engineering:** Transform and combine existing features creatively.
- **Documentation:** Keep track of your experiments and results. Document what works and what doesn't.

## Collaboration and Presentation

- **Collaboration:** Discuss your work openly within your team or with other teams. Sharing insights and learning from each other is encouraged.
- **Presentation:** Present your methodology, results, and the techniques that helped the most. Document your journey and the steps you took to achieve your results



In [24]:
import gdown
from pathlib import Path

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer


In [25]:
def download_from_gdrive(url, filename):
    # Extract the file ID from the URL
    file_id = url.split('/')[-2]
    download_url = f"https://drive.google.com/uc?id={file_id}"

    # Download the file
    if Path(filename).exists():
        print(f"File '{filename}' already exists. Skipping download.")
    else:
        gdown.download(download_url, filename, quiet=False)
        print(f"File downloaded as: {filename}")

train = 'https://drive.google.com/file/d/1guqSpDv1Q7ZZjSbXMYGbrTvGns0VCyU5/view?usp=drive_link'
valid = 'https://drive.google.com/file/d/1j7x8xhMimKbvW62D-XeDfuRyj9ia636q/view?usp=drive_link'
# Example usage

download_from_gdrive(train, 'train.csv')
download_from_gdrive(valid, 'valid.csv')

df = pd.read_csv('train.csv')
df_valid = pd.read_csv('valid.csv')

File 'train.csv' already exists. Skipping download.
File 'valid.csv' already exists. Skipping download.


<ipython-input-25-861d4f7287ed>:20: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


## Exploratory Data Analysis (EDA)

In [ ]:
#df.fiProductClassDesc.value_counts()

In [ ]:
#df.info()

In [ ]:
 #df.SalesID.nunique()

In [ ]:
# df.isnull().sum()

In [ ]:
#df.describe()

In [ ]:
#sns.histplot(data=df, x='SalePrice', bins=20)

In [ ]:
# to see value_counts for all categorical columns, but some realy categorical columns has numerical type like ModelID
categorical_cols = df.select_dtypes(exclude='number').columns
for col in categorical_cols:
  print(f"Value counts for column '{col}':")
  print(df[col].value_counts())
  print(f"NaN values:{df[col].isnull().sum()}")
  print()

Value counts for column 'UsageBand':
UsageBand
Medium    33985
Low       23620
High      12034
Name: count, dtype: int64
NaN values:331486

Value counts for column 'saledate':
saledate
2/16/2009 0:00    1932
2/15/2011 0:00    1352
2/19/2008 0:00    1300
2/15/2010 0:00    1219
2/11/2008 0:00    1100
                  ... 
1/16/2004 0:00       1
3/27/2006 0:00       1
7/25/2003 0:00       1
1/16/2006 0:00       1
6/9/2008 0:00        1
Name: count, Length: 3919, dtype: int64
NaN values:0

Value counts for column 'fiModelDesc':
fiModelDesc
310G        5039
416C        4869
580K        4315
310E        4233
140G        4083
            ... 
EX210-5        1
KX025          1
EX120-5F       1
EX100-5E       1
HW180          1
Name: count, Length: 4999, dtype: int64
NaN values:0

Value counts for column 'fiBaseModel':
fiBaseModel
580      19798
310      17354
D6       13110
416      12687
D5        9342
         ...  
830-2        1
272          1
PC230        1
KBD65        1
HW180        1


### 3. Data Preprocessing

In [26]:
def Num_to_Object(X, col):
    for col_ in col:
        X[col] = X[col].astype('object')
    return X

df = Num_to_Object(df, col = ['datasource'])
df_valid = Num_to_Object(df_valid, col = ['datasource'])

In [ ]:
def fix_mistakes(X, replacement_dict):
    for col, replacements in replacement_dict.items():
        X[col] = X[col].replace(replacements)
    return X

fix_mistakes(df, replacement_dict = {'Transmission': {'AutoShift': 'Autoshift'}})
fix_mistakes(df_valid, replacement_dict = {'Transmission': {'AutoShift': 'Autoshift'}})
#df['Transmission'] = df['Transmission'].replace('AutoShift', 'Autoshift')

In [28]:
def fix_data(X, date_col):
    for col in date_col:
        X[col] = pd.to_datetime(X[col])
        X[col + '_Year'] = X[col].dt.year
        X[col + '_Month'] = X[col].dt.month
        X = X.drop(col, axis=1)
    return X

df = fix_data(df, date_col = ['saledate'])
df_valid = fix_data(df_valid, date_col = ['saledate'])

In [ ]:
def first_word_name(X, col):
    for col_ in col:
        X[col_+'_first_word'] = X[col_].apply(lambda x: x.split()[0] if isinstance(x, str) else x)
    return X

first_word_name(df, col = ['fiProductClassDesc'])
first_word_name(df_valid, col = ['fiProductClassDesc'])

In [30]:
def ord_encod_nan(X, col, categories):
    encoder = OrdinalEncoder(categories=[categories], handle_unknown='use_encoded_value', unknown_value= -1)
    X[col + '_3'] = encoder.fit_transform(X[[col]])
    X[col + '_3'].replace(-1, np.nan, inplace=True)
    X[col + '_3'] = pd.to_numeric(X[col + '_3'], errors='coerce')

    return X


df = ord_encod_nan(df, col = 'UsageBand', categories = ['Low', 'Medium', 'High'])
df_valid = ord_encod_nan(df_valid, col = 'UsageBand', categories = ['Low', 'Medium', 'High'])

df = ord_encod_nan(df, col = 'ProductSize', categories = ['Mini', 'Compact', 'Small', 'Medium', 'Large / Medium', 'Large', 'High'])
df_valid = ord_encod_nan(df_valid, col = 'ProductSize', categories = ['Mini', 'Compact', 'Small', 'Medium', 'Large / Medium', 'Large', 'High'])

In [31]:
def replace_dict(X, col, repl_dict):
    new_name = col + '_3'

    X[new_name] = X[col]
    for old, new in repl_dict.items():
        X[new_name] = X[new_name].str.replace(old, new, regex=False)
    X[new_name].replace('None or Unspecified', -1)
    X[new_name].replace(-1, np.nan, inplace=True)
    X[new_name] = pd.to_numeric(X[new_name], errors='coerce')

    return X

df = replace_dict(df, col = 'Undercarriage_Pad_Width', repl_dict= {' inch': ''})
df_valid = replace_dict(df_valid, col= 'Undercarriage_Pad_Width', repl_dict = {' inch': ''})

df = replace_dict(df, col = 'Stick_Length', repl_dict= {"' ": '.', '"': ''})
df_valid = replace_dict(df_valid, col= 'Stick_Length', repl_dict = {"' ": '.', '"': ''})

In [ ]:
#df[df['YearMade'] == 1000].head(20) # i have not idias what to do with year 1000

## Mean / Target encoding    (depend on test / train split)

In [32]:
def make_target_mean_dict(X, y):
    X1 = X.copy()

    y1 = pd.DataFrame(y)
    X1 = pd.concat([X1, y1], axis=1)
    X1 = X1.rename(columns={0: 'SalePrice'})

    target_mean_dict = {}
    target_nan_mean_dict = {}
    for col in X1.select_dtypes(exclude='number').columns:
        target_mean_dict[col] = X1.groupby(col)['SalePrice'].mean().to_dict()
        target_nan_mean_dict[col] = X1[X1[col].isna()]['SalePrice'].mean()
    X1 = X1.drop(columns=['SalePrice'])

    return target_mean_dict, target_nan_mean_dict



def target_encode(X, target_mean_dict, target_nan_mean_dict):
    for col in X.select_dtypes(exclude='number').columns:
        X[col + '_2'] = X[col].map(target_mean_dict[col]).fillna(target_nan_mean_dict[col])
        X[col + '_2'] = X[col + '_2'].astype(float)
    return X

#target_mean_dict, target_nan_mean_dict = make_target_mean_dict(df, target_col = 'SalePrice')
#df = target_encode(df, target_mean_dict, target_nan_mean_dict)
#df_valid = target_encode(df_valid, target_mean_dict, target_nan_mean_dict)

# Testing model whith different parametrs

In [33]:
df2 = df.sample(400000, random_state=57)

y = df2['SalePrice']
X = df2.drop(columns=['SalePrice', 'SalesID', 'MachineID', 'ModelID'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=179)

target_mean_dict, target_nan_mean_dict = make_target_mean_dict(X_train, y_train)
X_train = target_encode(X_train, target_mean_dict, target_nan_mean_dict)
X_test = target_encode(X_test, target_mean_dict, target_nan_mean_dict)

X_train = X_train.select_dtypes('number')
X_test = X_test.select_dtypes('number')

X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())

## One single test

In [ ]:
# %%time

# model = RandomForestRegressor(n_jobs=-1,
#                               n_estimators = 300,
#                               #max_depth = 10,
#                               min_impurity_decrease = 10,
#                               random_state = 42,
#                               max_features = 8
#                               )
# model.fit(X_train, y_train)

# y_train_pred = model.predict(X_train)
# y_test_pred = model.predict(X_test)

# print(f'Train RMSE:', np.sqrt(mean_squared_error(y_train, y_train_pred)))
# print(f'Test RMSE:', np.sqrt(mean_squared_error(y_test, y_test_pred)))
# print(f'R²:' , r2_score(y_test, y_test_pred))
# print(f'Train MAE:', mean_absolute_error(y_train, y_train_pred))
# print(f'Test MAE:', mean_absolute_error(y_test, y_test_pred))


# Testing Grid of parametrs

In [34]:
%%time
param_grid = {
    'max_features': [5, 6, 7],
    'min_impurity_decrease': [5, 1000],
    #'n_estimators': [100, 200, 300]
}

model = RandomForestRegressor(n_jobs=-1,
                              n_estimators = 200,
                              #max_depth = 10,
                              #min_impurity_decrease = 10,
                              random_state = 57,
                              #max_features = 'sqrt'
                              )
model.fit(X_train, y_train)
# Настраиваем метрики для GridSearchCV
scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'R2': make_scorer(r2_score),
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False)
}

# Настраиваем GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring=scoring, refit='R2', return_train_score=True)
grid_search.fit(X_train, y_train)

# Получаем результаты всех тестов на сетке
results = grid_search.cv_results_

# Печатаем результаты тестов на всех параметрах сетки
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Params: {params} - Mean Test Score (negative MSE): {mean_score}")

# Получаем результаты всех тестов на сетке
results = grid_search.cv_results_

# Печатаем результаты тестов на всех параметрах сетки для каждой метрики
for mean_mse, mean_r2, mean_mae, params in zip(results['mean_test_MSE'], results['mean_test_R2'], results['mean_test_MAE'], results['params']):
    print(f"Params: {params} - Mean Test MSE: {mean_mse} - Mean Test R2: {mean_r2} - Mean Test MAE: {mean_mae}")

# Лучшие параметры и модель по метрике R2
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print(f"\nBest Parameters: {best_params}")

# Оценка лучшей модели на тестовом наборе
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"\nTest MSE of the Best Model: {test_mse:.2f}")
print(f"Test R2 of the Best Model: {test_r2:.2f}")
print(f"Test MAE of the Best Model: {test_mae:.2f}")

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
### Feature importance
# pd.Series(
#     model.feature_importances_,
#     index=model.feature_names_in_
# ).sort_values(ascending=False)

# Prediction on the Validation datas

In [ ]:
df2 = df
X_valid2 = df_valid

y = df2['SalePrice']
X = df2.drop(columns=['SalePrice', 'MachineID', 'ModelID', 'SalesID'])
X_valid = X_valid2.drop(columns=['MachineID', 'ModelID', 'SalesID'])

target_mean_dict, target_nan_mean_dict = make_target_mean_dict(X, y)
X = target_encode(X, target_mean_dict, target_nan_mean_dict)
X_valid = target_encode(X_valid, target_mean_dict, target_nan_mean_dict)

X = X.select_dtypes('number')
X_valid = X_valid.select_dtypes('number')

X = X.fillna(X.mean())
X_valid = X_valid.fillna(X.mean())

In [ ]:
%%time
model = RandomForestRegressor(n_jobs=-1,
                              n_estimators = 300,
                              #max_depth = 10,
                              min_impurity_decrease = 3,
                              random_state = 42,
                              max_features = 8,
                              #max_samples=0.75
                              )
model.fit(X, y)

y_valid_pred = model.predict(X_valid)

In [ ]:
# # Create a submission file
# submission = pd.DataFrame({'SalesID': df_valid['SalesID'], 'SalePrice': y_valid_pred})
# submission.to_csv('final_submission.csv', index=False)

### not used

In [ ]:
# scaler = MinMaxScaler()  -   nothing to chenge
# X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
# X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
# X_valid = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns, index=X_valid.index)

# feature_names = X_train.columns  is is aufully compicated
# imputer = SimpleImputer(strategy='mean')
# X_train = imputer.fit_transform(X_train)
# X_test = imputer.transform(X_test)
# X_valid = imputer.transform(X_valid)
# X_train = pd.DataFrame(X_train, columns=feature_names)
# X_test = pd.DataFrame(X_test, columns=feature_names)
# X_valid = pd.DataFrame(X_valid, columns=feature_names)

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(20, 20))
# sns.heatmap(X.corr(), vmin=-1, fmt=".1f", vmax=1, annot=True, cmap='BrBG')
# plt.show()